In [1]:
# Import the libraries

import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import random
from pandas.io.json import json_normalize
import os

## Authorization Flow

In [2]:
# Declare the credentials

CLIENT_ID = '6f17389130b3443f993c5e91f4cb6067'
CLIENT_SECRET = 'fff83d792abd40e9bd7285243284d8e2'
redirect_uri='http://localhost:7777/callback'
username = '22blfxzqxsdb7puge77ioamkq'

# Authorization flow
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Extract User's Top Medium Term Songs

In [3]:
directory = 'UserSongs'

In [4]:
all_songs = []

user_id = 0
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        user_songs = pd.read_csv(os.path.join(directory, filename))
        user_songs['user'] = user_id
        user_id += 1
        all_songs.append(user_songs)
        
all_songs = pd.concat(all_songs)
all_songs.reset_index(drop=True,inplace=True)
all_songs

,Unnamed: 0,artist,artist_uri,song,song_uri,duration_ms,explicit,album,popularity,user
0,0,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Science Fiction Double Feature (Glee Cast Vers...,spotify:track:4i3CKKl6S3fNkyGsgtJAss,267986,False,"Glee: The Music, The Rocky Horror Glee Show",42,0
1,1,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Sweet Transvestite (Glee Cast Version),spotify:track:5Uaz86QcbGZgTyS5sh6TmM,179920,False,"Glee: The Music, The Rocky Horror Glee Show",47,0
2,2,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Whatever Happened To Saturday Night? (Glee Cas...,spotify:track:3wZD98XkyFfkB2jHIORDnY,184400,False,"Glee: The Music, The Rocky Horror Glee Show",43,0
3,3,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Touch A Touch A Touch A Touch Me (Glee Cast Ve...,spotify:track:4QhUwFhOCM0qpORDogzvDs,149626,False,"Glee: The Music, The Rocky Horror Glee Show",49,0
4,4,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Time Warp (Glee Cast Version),spotify:track:1onMcvjW2nZ2dHtbJn2cN7,193560,False,"Glee: The Music, The Rocky Horror Glee Show",59,0
...,...,...,...,...,...,...,...,...,...,...
145,45,They Might Be Giants,spotify:artist:6zB02lwP6L6ZH32nggQiJT,Istanbul (Not Constantinople),spotify:track:63vL5oxWrlvaJ0ayNaQnbX,153813,False,Flood,67,2
146,46,O Grilo,spotify:artist:22KEpOwThQ5q1DGochfayO,Serenata Existencialista,spotify:track:1tIJba7S9uHsNml4BqZrah,163209,False,HerÃ³i do Futuro,47,2
147,47,Supercombo,spotify:artist:73HkjgziMO6I83vFOS8mo1,Todo Dia Ã‰ Dia de Comemorar,spotify:track:2TCAdLDCBwiNU7BdGcX8NE,213103,False,Todo Dia Ã‰ Dia de Comemorar,41,2
148,48,Selvagens Ã Procura de Lei,spotify:artist:093ybF4c81CndQe6qWtc8S,Gostar SÃ³ Dela,spotify:track:0FJCzNONkZwJurGiQYYmoH,187468,False,Gostar SÃ³ Dela,35,2


## Extract Users' Top 50 Tracks' Audio Features

In [5]:
audio_feat = []
for song in all_songs['song_uri']:
    row = pd.DataFrame(sp.audio_features(tracks=[song]))
    audio_feat.append(row)
audio_feat = pd.concat(audio_feat)

audio_feat.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.428,0.559,0,-7.052,1,0.0246,0.2820,0.000000,0.382,0.361,90.198,audio_features,4i3CKKl6S3fNkyGsgtJAss,spotify:track:4i3CKKl6S3fNkyGsgtJAss,https://api.spotify.com/v1/tracks/4i3CKKl6S3fN...,https://api.spotify.com/v1/audio-analysis/4i3C...,267987,4
0,0.736,0.571,4,-3.942,1,0.0457,0.4510,0.000618,0.223,0.554,107.788,audio_features,5Uaz86QcbGZgTyS5sh6TmM,spotify:track:5Uaz86QcbGZgTyS5sh6TmM,https://api.spotify.com/v1/tracks/5Uaz86QcbGZg...,https://api.spotify.com/v1/audio-analysis/5Uaz...,179920,4
0,0.566,0.913,3,-4.607,1,0.0577,0.1180,0.000005,0.194,0.961,164.982,audio_features,3wZD98XkyFfkB2jHIORDnY,spotify:track:3wZD98XkyFfkB2jHIORDnY,https://api.spotify.com/v1/tracks/3wZD98XkyFfk...,https://api.spotify.com/v1/audio-analysis/3wZD...,184400,4
0,0.309,0.789,10,-6.232,0,0.0457,0.2610,0.000068,0.163,0.606,170.713,audio_features,4QhUwFhOCM0qpORDogzvDs,spotify:track:4QhUwFhOCM0qpORDogzvDs,https://api.spotify.com/v1/tracks/4QhUwFhOCM0q...,https://api.spotify.com/v1/audio-analysis/4QhU...,149627,4
0,0.395,0.817,11,-5.757,0,0.1250,0.0442,0.000072,0.243,0.635,176.196,audio_features,1onMcvjW2nZ2dHtbJn2cN7,spotify:track:1onMcvjW2nZ2dHtbJn2cN7,https://api.spotify.com/v1/tracks/1onMcvjW2nZ2...,https://api.spotify.com/v1/audio-analysis/1onM...,193560,4


## Data Cleaning

In [6]:
# Drop unnecessary features

audio_feat.drop(['type','track_href','analysis_url','time_signature','duration_ms','uri','instrumentalness','liveness','loudness','key','mode'],1,inplace=True)
audio_feat.set_index('id',inplace=True)
audio_feat.head()

,danceability,energy,speechiness,acousticness,valence,tempo
id,,,,,,
4i3CKKl6S3fNkyGsgtJAss,0.428,0.559,0.0246,0.2820,0.361,90.198
5Uaz86QcbGZgTyS5sh6TmM,0.736,0.571,0.0457,0.4510,0.554,107.788
3wZD98XkyFfkB2jHIORDnY,0.566,0.913,0.0577,0.1180,0.961,164.982
4QhUwFhOCM0qpORDogzvDs,0.309,0.789,0.0457,0.2610,0.606,170.713
1onMcvjW2nZ2dHtbJn2cN7,0.395,0.817,0.1250,0.0442,0.635,176.196


In [7]:
# Normalize tempo feature

columns = ['danceability','energy','speechiness','acousticness','valence','tempo']
scaler = MinMaxScaler()
scaler.fit(audio_feat[columns])
audio_feat[columns] = scaler.transform(audio_feat[columns])
audio_feat.head()

,danceability,energy,speechiness,acousticness,valence,tempo
id,,,,,,
4i3CKKl6S3fNkyGsgtJAss,0.273128,0.547492,0.000000,0.284468,0.346631,0.142719
5Uaz86QcbGZgTyS5sh6TmM,0.725404,0.560299,0.051665,0.455025,0.554717,0.293531
3wZD98XkyFfkB2jHIORDnY,0.475771,0.925293,0.081048,0.118957,0.993531,0.783898
4QhUwFhOCM0qpORDogzvDs,0.098385,0.792956,0.051665,0.263274,0.610782,0.833035
1onMcvjW2nZ2dHtbJn2cN7,0.224670,0.822839,0.245837,0.044477,0.642049,0.880045


In [8]:
# Check out features summary

audio_feat.describe()

,danceability,energy,speechiness,acousticness,valence,tempo
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,0.452129,0.607663,0.075067,0.360588,0.483589,0.427966
std,0.211865,0.261417,0.127596,0.307801,0.260224,0.231213
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.285977,0.443170,0.018609,0.052980,0.269272,0.257641
50%,0.472834,0.664354,0.039545,0.273367,0.469003,0.450487
75%,0.602056,0.818303,0.072968,0.614733,0.691375,0.548335
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
#Making into A CSV

audio_feat.to_csv('AudioFeatures.csv')

## Building the Song Recommender

### K-Means Cluster Analysis

In [10]:
# Get 20 clusters from 100 songs

clusters = user_id*10
kmeans = KMeans(n_clusters=clusters)
kmeans.fit(audio_feat)
len(kmeans.labels_)

150

In [11]:
from sklearn.decomposition import PCA 

pca = PCA(3) 
pca.fit(audio_feat) 
  
pca_data = pd.DataFrame(pca.transform(audio_feat)) 
  
print(pca_data.head())

          0         1         2
0  0.070657  0.180337 -0.309505
1  0.117911 -0.262679 -0.098267
2 -0.591124 -0.261636  0.318017
3 -0.321364  0.215688  0.373750
4 -0.511728  0.176882  0.363348


In [12]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(audio_feat)
y_kmeans = kmeans.fit_predict(scaled)
y_kmeans

array([28, 14, 27, 16,  2, 27,  7,  0, 14, 22,  0,  5,  6,  3,  0,  0, 12,
        5, 25, 20,  0,  5, 16,  6, 14, 25,  0, 27,  6, 21,  1,  4, 14, 14,
       10, 10, 25, 11,  5, 27,  0,  6,  5, 16, 20,  3,  9, 14,  5,  1, 22,
        3,  4, 13, 13,  2, 13, 19, 16,  1, 16, 19, 28,  3, 20, 18, 17, 11,
       10, 26, 20, 11,  3, 24, 19, 11, 20, 17,  9, 25, 22, 19, 14, 25,  7,
        1, 11,  2,  7, 12,  4, 19, 11, 18, 11, 17,  7,  4, 26, 20, 23, 12,
       23, 22,  2, 10, 10, 27, 10, 23, 13, 10, 29, 27, 23, 15, 23,  1, 15,
       14, 10, 12, 20, 22,  9, 19, 25,  8,  6,  6, 20, 20,  3, 28, 10, 18,
       23,  6, 27,  2,  6,  9, 10, 17, 22,  5, 10,  2,  0, 16])

In [13]:
# Updating dataframe with assigned clusters 

audio_feat['cluster'] = y_kmeans
audio_feat['artist'] = all_songs.artist.tolist()
audio_feat['title'] = all_songs.song.tolist()
audio_feat['user'] = all_songs.user.tolist()
audio_feat.head()

,danceability,energy,speechiness,acousticness,valence,tempo,cluster,artist,title,user
id,,,,,,,,,,
4i3CKKl6S3fNkyGsgtJAss,0.273128,0.547492,0.000000,0.284468,0.346631,0.142719,28,Glee Cast,Science Fiction Double Feature (Glee Cast Vers...,0
5Uaz86QcbGZgTyS5sh6TmM,0.725404,0.560299,0.051665,0.455025,0.554717,0.293531,14,Glee Cast,Sweet Transvestite (Glee Cast Version),0
3wZD98XkyFfkB2jHIORDnY,0.475771,0.925293,0.081048,0.118957,0.993531,0.783898,27,Glee Cast,Whatever Happened To Saturday Night? (Glee Cas...,0
4QhUwFhOCM0qpORDogzvDs,0.098385,0.792956,0.051665,0.263274,0.610782,0.833035,16,Glee Cast,Touch A Touch A Touch A Touch Me (Glee Cast Ve...,0
1onMcvjW2nZ2dHtbJn2cN7,0.224670,0.822839,0.245837,0.044477,0.642049,0.880045,2,Glee Cast,Time Warp (Glee Cast Version),0


In [14]:
# Removing clusters that only have one song in them

delete_clusters = []
cluster = 0
while cluster < (len(audio_feat.cluster.unique())-1):
    if audio_feat.groupby('cluster').count().loc[cluster].danceability == 1:
        delete_clusters.append(cluster)
        
    cluster+=1
    
delete_clusters

[8, 21, 24]

In [30]:
audio_feat.groupby(['cluster','user']).count()

danceability  energy  speechiness  acousticness  valence  tempo  \
cluster user                                                                    
0       0                7       7            7             7        7      7   
        2                1       1            1             1        1      1   
1       0                2       2            2             2        2      2   
        1                2       2            2             2        2      2   
        2                1       1            1             1        1      1   
...                    ...     ...          ...           ...      ...    ...   
27      2                3       3            3             3        3      3   
28      0                1       1            1             1        1      1   
        1                1       1            1             1        1      1   
        2                1       1            1             1        1      1   
29      2                1       1            1             1        1      1   

              artist  title  
cluster user                 
0       0          7      7  
        2          1      1  
1       0          2      2  
        1          2      2  
        2          1      1  
...              ...    ...  
27      2          3      3  
28      0          1      1  
        1          1      1  
        2          1      1  
29      2          1      1  

[65 rows x 8 columns]

In [16]:
audio_feat.reset_index(inplace=True)

In [17]:
i = 0
while i < (len(audio_feat.cluster.unique())-1):
    if audio_feat.loc[[i]].cluster.tolist()[0] in delete_clusters:
        audio_feat.drop(i,0,inplace=True)
    i+=1

In [18]:
audio_feat.set_index('id',inplace=True)

In [19]:
# Create list of lists of song ids to put into recommendation function
i=0
list_of_recs = [0]*len(audio_feat.groupby('cluster').count())
while i<len(audio_feat.groupby('cluster').count()):
    list_of_recs[i] = audio_feat.loc[audio_feat['cluster'] == i].index.tolist()
    i+=1

list_of_recs = [ele for ele in list_of_recs if ele != []] 
len(list_of_recs)

30

In [20]:
# Adjust list for clusters so that each cluster has a maximum of 5 seed songs

j = 0
adj_list_of_recs = [0]*len(list_of_recs)
while j<len(list_of_recs):
    if 0 < len(list_of_recs[j]) < 6:
        adj_list_of_recs[j] = list_of_recs[j]
    elif len(list_of_recs[j]) > 5:
        adj_list_of_recs[j] = random.sample(list_of_recs[j], 5)
    j += 1

len(adj_list_of_recs)

30

In [21]:
#Getting 1 recommended song from each cluster with less than 4 songs, 2 recommended songs from each cluster with 4-5 songs

k = 0
list_of_recommendations = [0]*len(list_of_recs)
while k < len(list_of_recs):
    if len(adj_list_of_recs[k]) < 4:
        list_of_recommendations[k] = sp.recommendations(seed_tracks=adj_list_of_recs[k],limit=1)
    else:
        list_of_recommendations[k] = sp.recommendations(seed_tracks=adj_list_of_recs[k],limit=2)
    k += 1
    
pd.json_normalize(list_of_recommendations[15], record_path='tracks').id

0    1qwamT1qN521FuTtOaOt4P
Name: id, dtype: object

In [22]:
list_of_recommendations_converted = [0]*len(list_of_recs)

l = 0
while l < len(list_of_recs):
    list_of_recommendations_converted.append(pd.json_normalize(list_of_recommendations[l], record_path='tracks').id.tolist())
    l += 1

no_integers = [x for x in list_of_recommendations_converted if not isinstance(x, int)]
list_of_recommendations_converted = [item for elem in no_integers for item in elem]

len(list_of_recommendations_converted)

52

In [23]:
# Authorization flow

scope = "playlist-modify-public"
token = util.prompt_for_user_token(username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [24]:
# Create new playlist and insert it straight to user's library

def create_playlist(sp, username, playlist_name, playlist_description):
    playlists = sp.user_playlist_create(username, playlist_name, description = playlist_description)

In [25]:
create_playlist(sp, username, 'Sony Play - Vi+Gian+Dani', 'Choose a friend to discover brand new music with. We create an adventurous playlist curated to both of your tastes!')

In [26]:
# Fetch user's playlist library

def fetch_playlists(sp, username):
    """
    Returns the user's playlists.
    """
        
    id = []
    name = []
    num_tracks = []
    
    # Make the API request
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        id.append(playlist['id'])
        name.append(playlist['name'])
        num_tracks.append(playlist['tracks']['total'])

    # Create the final df   
    df_playlists = pd.DataFrame({"id":id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [27]:
# Find the new playlist's id

fetch_playlists(sp,username).head(3)

,id,name,#tracks
0,1nuFUzDrvl1EMYKDCbcKMY,Sony Play - Vi+Gian+Dani,0
1,3TSfB52P3BSXqXnWYlKD9T,Spotify Discover Together,32
2,1dLIGzCAAvrS73FGayI0zf,Loka de Pinga – Kika Boom,7


In [28]:
extracted_id = fetch_playlists(sp,username).id[0]

In [29]:
# Finally, fill the new playlist with the recommended songs straight to the user's library!

sp.user_playlist_add_tracks(username, extracted_id, list_of_recommendations_converted, position=None)

{'snapshot_id': 'Myw4OGMyMDJmODhhNzZjNmRjMTVkYjNmOWNhNmE0OTQyMjEyZGM5YWZl'}